# Write python flask marshmallow schemas for api endpoints automatically

#### Save the schemas as python classes or python dicts given the api base url and the api enpoints

### API directory 

In [1]:
cd ../../../../Apps/Python/bolsao-api

C:\Users\luisr\Desktop\Repositories\Apps\Python\bolsao-api


### Import modules

In [2]:
import requests, json, pandas as pd, numpy as np
from IPython.display import clear_output as co
from urllib.parse import urlencode

C:\Users\luisr\AppData\Roaming\Python\Python38\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


### Functions to build code for schemas as python classes or dicts

In [3]:
template_class = """class {}(Schema):
{}
"""

template_fields_class = "    {} = {}({}metadata={})\n"

template_dict = """{} = Mark0
{}
Mark1
"""

template_fields_dict = '    "{}": {}({}metadata={}),\n'

datatypes = {'str': 'String', 'int': 'Integer', 'bool': 'Boolean', 'float': 'Float', 'list': 'List', 'dict': 'Dict'}

def get_nested_dtypes(values):
    subtype = datatypes[type(values[0]).__name__]
    nested_subtype = '' if subtype != 'List' else (get_nested_dtypes(values[0]))
    return f'{subtype}({nested_subtype})'

cap_map = lambda s: s.capitalize()
cap_field_map = lambda field: ' '.join(list(map(cap_map, field.split('_'))))
cap_key_map = lambda key: ''.join(list(map(cap_map, key.split('/')[1:])))

fields_dtypes = {
    'str': [
        'blockingAlertUuid', 'startNode',
        'causeType', 'reportDescription'
    ],
    'float': ['nThumbsUp'],
    'dict': ['causeAlert'],
    'list': ['anexos']
}

field_subtypes = {
    'anexos': ['String()']
}

def build_class_schemas(responses):
    schemas = {key: {'class': cap_key_map(key)} for key in responses};
    classes = {}
    for key in responses:
        str_fields = []
        if len(responses[key]):
            for field, value in responses[key][0].items():
                dtype = type(value).__name__
                type_params = 'validate=OneOf([0, 1]), ' if 'status' and 'status_count' not in field else in field else ('format="date-time", ' if 'timestamp' in field else '')
                type_params = f'data_key="{field}", allow_none=True, ' + type_params
                if dtype == 'list':
                    if 'ids' in field or not len(value):
                        if field == 'waze_flood_ids': subtype = 'String()'
                        else: subtype = 'Integer()'
                    else: subtype = get_nested_dtypes(value)
                    type_params = f'{subtype}, ' + type_params
                is_number = type(value) is int or type(value) is float
                example = (value if not is_number else (None if np.isnan(value) or value is None else value))
                fieldname = field.replace(' ', '_').replace('-', '_').replace('___', '_')
                # mannualy input the right data type for fields with None
                if dtype == 'NoneType':
                    for field_dtype in fields_dtypes:
                        if field in fields_dtypes[field_dtype]:
                            dtype = field_dtype
                            if field in field_subtypes:
                                type_params = f'{field_subtypes[field]}, {type_params}'
                datatype = datatypes[dtype]
                metadata = str({'data_key': field, 'title': cap_field_map(field), 'example': example})
                str_field = template_fields_class.format(fieldname, datatype, type_params, metadata)
                str_fields.append(str_field)
            classes[key] = template_class.format(schemas[key]['class'], ''.join(str_fields)).replace('Mark0', '{').replace('Mark1', '}')
    return classes

def build_dict_schemas(responses):
    schemas = {key: {'class': cap_key_map(key)} for key in responses};
    classes = {}
    for key in responses:
        str_fields = []
        if len(responses[key]):
            for field, value in responses[key][0].items():
                dtype = type(value).__name__
                type_params = 'validate=OneOf([0, 1]), ' if 'status' in field else ('format="date-time", ' if 'timestamp' in field else '')
                type_params = f'data_key="{field}", allow_none=True, ' + type_params
                if dtype == 'list':
                    if 'ids' in field or not len(value): subtype = 'String()'
                    else: subtype = get_nested_dtypes(value)
                    type_params = f'{subtype}, ' + type_params
                if dtype == 'NoneType':
                    for field_dtype in fields_dtypes:
                        if field in fields_dtypes[field_dtype]:
                            dtype = field_dtype
                            if field in field_subtypes:
                                type_params = f'{field_subtypes[field]}, {type_params}'
                metadata = str({'datakey': field, 'title': cap_field_map(field), 'example': value})
                str_field = template_fields_dict.format(field, datatypes[dtype], type_params, metadata)
                str_fields.append(str_field)
            classes[key] = template_dict.format(schemas[key]['class'], ''.join(str_fields)).replace('Mark0', '{').replace('Mark1', '}')
    return classes

### Load api spec and get api endpoints

In [11]:
# spec = json.load(open('openapi.json', 'r'))
spec = requests.get('http://127.0.0.1:5000/openapi.json').json()

paths = list(spec['paths'].keys())
paths_params = [path for path in paths if '{' in path]
paths_history = [path for path in paths if 'history' in path]
paths_cam = [path for path in paths if 'cam' in path and 'cameras' not in path]
paths_webapps = [path for path in paths if 'flood-models' in path or 'octa' in path] # endpoints with parameters

paths = set(paths)
for pathlist in [paths_params, paths_history, paths_cam, paths_webapps]:
    paths = paths.difference(pathlist)
paths = list(paths)

### Define history endpoints with respective query parameters

In [5]:
query_open = {'at': '2023-02-07 20:00:00'}
query_window = {'start': '2023-02-07 20:00:00', 'end': '2023-02-07 20:30:00'}
query_history = {'start': '2023-02-07 20:00:00', 'end': '2023-02-07 20:30:00', 'freq': '15Min'}
query_history_summary = {'start': '2023-02-07 20:00:00', 'end': '2023-02-07 21:00:00', 'freq': '1H', 'summary': 'True'}

query_open = urlencode(query_open)
query_window = urlencode(query_window)
query_history = urlencode(query_history)
query_history_summary = urlencode(query_history_summary)

paths_history_query = [
    f'/comando/api/history?{query_window}',
    f'/comando/api/history/open?{query_open}',
    f'/city/history?{query_history}',
    f'/city/history?{query_history_summary}',
    f'/city/history/window?{query_window}',
    f'/city/history/open?{query_open}',
    f'/polygons/history?{query_history}',
    f'/polygons/history?{query_history_summary}',
    f'/polygons/history/window?{query_window}',
    f'/polygons/history/open?{query_open}',
    f'/polygons/overview/history/window?{query_window}',
    f'/polygons/overview/history/open?{query_open}',
    f'/ipp/polygons/history?{query_history}',
    f'/ipp/polygons/history?{query_history_summary}',
    f'/ipp/polygons/history/window?{query_window}',
    f'/ipp/polygons/history/open?{query_open}',
    f'/ipp/polygons/overview/history/window?{query_window}',
    f'/ipp/polygons/overview/history/open?{query_open}',
]

### Get json response for each endpoint

In [122]:
baseurl = 'https://bolsao-api-j2fefywbia-rj.a.run.app'
# baseurl = 'http://127.0.0.1:5000'

pathlist = paths + paths_history_query

responses, fail = {}, {}
for endpoint in pathlist:
    try:
        co(True); print(f'Requesting: {pathlist.index(endpoint)+1}/{len(pathlist)}, Fail: {len(fail)}/{len(pathlist)} - {endpoint}')
        if len(fail): print(f'\nFailed: {fail}')
        key = endpoint.split('?')[0] if '?' in endpoint else endpoint
        res = requests.get(f'{baseurl}{endpoint}')
        if res:
            if key in responses: responses[key][0] = {**res.json()[0], **responses[key][0]}
            else: responses[key] = res.json()
        else: raise Exception(f'Error Code: {res.status_code}')
    except Exception as err:
        fail[endpoint] = str(err)

Requesting: 41/41, Fail: 1/41 - /ipp/polygons/overview/history/open?at=2023-02-07+20%3A00%3A00

Failed: {'/stations/alertario': 'Error Code: 404'}


### Evaluate responses

In [123]:
print('Failed:', len(fail))
print('\nErrors:', fail, '\n')

display(pd.DataFrame(
    [[len(res), (len(res[0]) if type(res) is list else np.nan), type(res).__name__]  for res in responses.values()],
    responses.keys(), ['Response Size', 'Response Fields', 'Response Type']
))

Failed: 1

Errors: {'/stations/alertario': 'Error Code: 404'} 



,Response Size,Response Fields,Response Type
/clusters,79,49,list
/comando/events,36,16,list
/polygons/static,79,21,list
/ipp/polygons,968,38,list
/waze/jams,131,18,list
/stations/inmet,4,24,list
/predict,5,10,list
/predictions,22265,10,list
/polygons/alertario,79,40,list
/polygons,79,49,list


### Build class and dict schemas for the auto openai documentation with APIFlask framework

In [78]:
classes = build_class_schemas(responses)

dicts = build_dict_schemas(responses)

### Write python files with the dict schemas code

In [75]:
import_statement = """
from apiflask import Schema
from apiflask.fields import Integer, String, Float, Boolean, List, Dict
from apiflask.validators import OneOf

"""

schemas_text = "\n".join(dicts.values())

# UNCOMMENT TO SAVE NEW SCHEMAS
# file_schema = open('modules/schemas_dicts.py', 'w', encoding='utf-8')

# file_schema.write(import_statement)
# file_schema.write(schemas_text)
# file_schema.close()

In [ ]:
print("\n".join(dicts.values()))

### Write python files with the class schemas code

In [76]:
import_statement = """
from apiflask import Schema
from apiflask.fields import Integer, String, Float, Boolean, List, Dict
from apiflask.validators import OneOf

"""

schemas_text = "\n".join(classes.values())

# UNCOMMENT TO SAVE NEW SCHEMAS
# file_schema = open('modules/schemas_classes.py', 'w', encoding='utf-8')

# file_schema.write(import_statement)
# file_schema.write(schemas_text)
# file_schema.close()

In [ ]:
print("\n".join(classes.values()))

### Write python statements to import the schema classes or dicts

In [64]:
paths_cap = list(map(cap_key_map, responses.keys()))

schema_import_statement = f"""
from modules.schemas_classes import {', '.join(paths_cap)}

"""

print(schema_import_statement)


from modules.schemas_classes import Clusters, ComandoEvents, PolygonsStatic, IppPolygons, WazeJams, StationsInmet, Predict, Predictions, PolygonsAlertario, Polygons, WazeIrregularities, WazeAlerts, CamerasAlertarioStations, City, PredictLive, Cameras, PolygonsOverview, ClustersStatic, IppPolygonsOverview, ClustersOverview, PolygonsAlertarioStations, StationsAlertario, StationsAlertarioApi, ComandoApiHistory, ComandoApiHistoryOpen, CityHistory, CityHistoryWindow, CityHistoryOpen, PolygonsHistory, PolygonsHistoryWindow, PolygonsHistoryOpen, PolygonsOverviewHistoryWindow, PolygonsOverviewHistoryOpen, IppPolygonsHistory, IppPolygonsHistoryWindow, IppPolygonsHistoryOpen, IppPolygonsOverviewHistoryWindow, IppPolygonsOverviewHistoryOpen




### Write python decorators to set the endpoints response schemas 

In [224]:
app_output_template = """@app.output({}(many=True))\n"""

print('\n'.join([app_output_template.format(title) for title in paths_cap]))

@app.output(City(many=True))

@app.output(Polygons(many=True))

@app.output(StationsAlertarioApi(many=True))

@app.output(Cameras(many=True))

@app.output(IppPolygons(many=True))

@app.output(WazeAlerts(many=True))

@app.output(Predict(many=True))

@app.output(ComandoEvents(many=True))

@app.output(Predictions(many=True))

@app.output(WazeJams(many=True))

@app.output(StationsInmet(many=True))

@app.output(ClustersOverview(many=True))

@app.output(ClustersStatic(many=True))

@app.output(CamerasAlertarioStations(many=True))

@app.output(PolygonsStatic(many=True))

@app.output(Clusters(many=True))

@app.output(PolygonsAlertario(many=True))

@app.output(IppPolygonsOverview(many=True))

@app.output(PolygonsAlertarioStations(many=True))

@app.output(PredictLive(many=True))

@app.output(StationsAlertario(many=True))

@app.output(PolygonsOverview(many=True))

@app.output(ComandoApiHistory(many=True))

@app.output(ComandoApiHistoryOpen(many=True))

@app.output(CityHistory(many=True))

@app.out